# Differential Equations – Convergence

1. Discretize the `sin` function with `n` points. Calculate its derivative at these points using finite differences. Study the convergence of this method assuming you know the solution (`sin' = cos`). Calculate and plot the convergence order `p`.

2. Repeat the above, but assume you don't know the solution ahead of time, i.e. study the self convergence.

3. Use Richardson extrapolation to approximate the true derivative.

4. Repeat (some of) the above using the advection equation from Geoffrey Ryan's notebook. Pick a particular time (e.g. `t = 0.5`) and apply the convergence tests there.

In [1]:
function discretize(f, xmin, xmax, npoints)
    x = LinRange(xmin, xmax, npoints)
    y = f.(x)
    return x, y
end

discretize (generic function with 1 method)

In [2]:
x9, sin9 = discretize(sin, 0, 2π, 9)

(LinRange{Float64}(0.0, 6.283185307179586, 9), [0.0, 0.7071067811865475, 1.0, 0.7071067811865476, 1.2246467991473532e-16, -0.7071067811865475, -1.0, -0.7071067811865477, -2.4492935982947064e-16])

In [3]:
function deriv(y, h)
    dy = similar(y)
    # Do nothing useful at the boundaries
    dy[1] = NaN
    for i in 2:length(y)-1
        # Centred finite difference, second order accurate
        dy[i] = (y[i+1] - y[i-1]) / 2h
    end
    dy[end] = NaN
    return dy
end

deriv (generic function with 1 method)

In [4]:
deriv(sin9, 2π/8)

9-element Vector{Float64}:
 NaN
   0.6366197723675814
   7.067899292141149e-17
  -0.6366197723675813
  -0.9003163161571061
  -0.6366197723675815
  -1.4135798584282297e-16
   0.6366197723675813
 NaN

In [5]:
# Choose two resolutions
n1 = 11
n2 = 21
h1 = 2π/(n1-1)
h2 = 2π/(n2-1)
h1, h2

(0.6283185307179586, 0.3141592653589793)

In [6]:
# Discretize `sin` twice
x_h1, sin_h1 = discretize(sin, 0, 2π, n1)
x_h2, sin_h2 = discretize(sin, 0, 2π, n2);

In [7]:
deriv_sin_h1 = deriv(sin_h1, h1)
deriv_sin_h2 = deriv(sin_h2, h2);

In [8]:
using WGLMakie

In [9]:
fig = Figure()
ax = Axis(fig[1, 1])
lines!(x_h1, sin_h1, color=:red)
lines!(x_h2, sin_h2, color=:green)
plot!(x_h1, deriv_sin_h1, color=:blue)
plot!(x_h2, deriv_sin_h2, color=:pink)
fig

In [10]:
# Plot error
fig = Figure()
ax = Axis(fig[1, 1])
plot!(x_h1, deriv_sin_h1 - cos.(x_h1), color=:blue)
plot!(x_h2, deriv_sin_h2 - cos.(x_h2), color=:pink)
fig

In [11]:
# Resample the finer grid on the coarse grid points
x_h2_resampled = x_h2[1:2:end]
deriv_sin_h2_resampled = deriv_sin_h2[1:2:end];

In [12]:
# Plot ratio of errors
fig = Figure()
ax = Axis(fig[1, 1])
plot!(x_h1, (deriv_sin_h1 - cos.(x_h1)) ./ (deriv_sin_h2_resampled - cos.(x_h2_resampled)), color=:blue)
fig

In [13]:
# Calculate convergence order
p = log2.((deriv_sin_h1 - cos.(x_h1)) ./ (deriv_sin_h2_resampled - cos.(x_h2_resampled)));

In [14]:
# Plot ratio of errors
fig = Figure()
ax = Axis(fig[1, 1])
plot!(x_h1, p, color=:blue)
fig